In [ ]:
pip install -U langchain-community


In [ ]:
pip install flask langchain openai beautifulsoup4 requests



In [ ]:
pip install faiss-cpu

In [ ]:
pip install sentence-transformers


In [ ]:
pip install flask langchain openai faiss-cpu sentence-transformers beautifulsoup4 requests python-dotenv


In [ ]:
pip install faiss-cpu


In [ ]:
pip install gunicorn pyngrok


In [ ]:
pip install flask selenium webdriver-manager langchain faiss-cpu transformers pyngrok


In [ ]:
!sudo apt install -y chromium-browser

In [ ]:
pip install python-dotenv

In [ ]:
# api key - sk-proj-7zM7fxSz893IEeP481o7Kk2yEcY0MJnKqC9cg7wqT51wM930pdzDVILvChsgvl-CEk4CKLxapRT3BlbkFJDnt2jTZblgH5vBSAFMxq2lK6BwE0G4_K5uEupwrTxeHhSmjn_dKi8h0_nYsAA
# URL = "https://brainlox.com/courses/category/technical"
# ngrok_auth_token = "2sLHxLr6pR4N5vBZwV2XB33udBd_4683LAcSLYnqkpJ1k8esx"

In [ ]:
import os
import json
from flask import Flask, request, jsonify
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

# ✅ Securely load OpenAI API key
openai_api_key = "sk-proj-7zM7fxSz893IEeP481o7Kk2yEcY0MJnKqC9cg7wqT51wM930pdzDVILvChsgvl-CEk4CKLxapRT3BlbkFJDnt2jTZblgH5vBSAFMxq2lK6BwE0G4_K5uEupwrTxeHhSmjn_dKi8h0_nYsAA"

if not openai_api_key:
    raise ValueError("❌ OpenAI API key is missing.")
else:
    print("✅ OpenAI API Key loaded successfully.")

# ✅ Step 1: Extract and Validate Data from Brainlox
URL = "https://brainlox.com/courses/category/technical"

try:
    loader = WebBaseLoader(URL)
    raw_documents = loader.load()

    if not raw_documents:
        raise ValueError("❌ No documents loaded. The website might require JavaScript rendering.")

    # Ensure we load at least 10 documents
    documents = [
        Document(page_content=doc.page_content, metadata={"source": doc.metadata.get("source", URL)})
        for doc in raw_documents[:10]  # Only take first 10 docs
    ]

    print(f"✅ Loaded {len(documents)} documents from {URL}")

except Exception as e:
    print(f"❌ Error loading documents: {e}")
    documents = []  # Ensure documents is always defined

if not documents:
    raise ValueError("❌ No valid documents found. Try another scraping method.")

# ✅ Step 2: Initialize FAISS Vector Store
try:
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    texts = [doc.page_content for doc in documents]

    # Store in FAISS
    db = FAISS.from_texts(texts, embeddings)
    db.save_local("faiss_index")  # ✅ Save FAISS index
    print("✅ FAISS index created and saved.")

    db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    retriever = db.as_retriever()
    print("✅ FAISS vector store loaded successfully.")

except Exception as e:
    print(f"❌ Error initializing FAISS: {e}")
    raise

# ✅ Step 3: Initialize LangChain Chat Model
try:
    llm = OpenAI(api_key=openai_api_key, temperature=0.5)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )

    print("✅ LangChain Chat Model initialized successfully.")

except Exception as e:
    print(f"❌ Error initializing ConversationalRetrievalChain: {e}")
    chain = None

# ✅ Step 4: Deploy Flask Chatbot with Ngrok
from pyngrok import ngrok

ngrok.set_auth_token("2sLHxLr6pR4N5vBZwV2XB33udBd_4683LAcSLYnqkpJ1k8esx")

public_url = ngrok.connect(5000)
print(f"🌍 Public URL: {public_url}")

# ✅ Step 5: Initialize Flask App
app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chatbot_response():
    """Handles chatbot conversation."""
    if not chain:
        return jsonify({"error": "Chatbot is disabled. Set up an OpenAI API key or use a local LLM."}), 400

    data = request.json
    user_input = data.get("message")

    if not user_input:
        return jsonify({"error": "Message is required"}), 400

    try:
        response = chain.run({"question": user_input})
        return jsonify({
            "response": response,
            "chat_history": memory.chat_memory.messages  # ✅ Returns conversation history
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    print(f"🚀 Chatbot server is running on: {public_url}")
    app.run(host="0.0.0.0", port=5000, debug=True)  # ✅ Debug Mode ON
